In [1]:
import pandas as pd
import numpy as np
import time
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
def timestamp_datetime(value):
    format = '%Y-%m-%d %H:%M:%S'
    dt = time.localtime(value)
    dt = time.strftime(format, dt)
    dt = datetime.datetime.strptime(dt, format)
    return dt

In [3]:
def convert_timestamp(data):
    data['time'] = data.context_timestamp.apply(timestamp_datetime)
    data['day'] = data.time.apply(lambda x: int(x.day))
    data['hour'] = data.time.apply(lambda x: int(x.hour))
    # 统计每个用户每天浏览淘宝次数
    user_query_day = data.groupby(['user_id', 'day']).size().reset_index().rename(columns={0: 'user_queryy_day'})
    data = pd.merge(data, user_query_day, 'left', on=['user_id', 'day'])
    # 统计每个用户在每天和每个小时浏览淘宝次数
    user_query_day_hour = data.groupby(['user_id', 'day', 'hour']).size().reset_index()\
                            .rename(columns={0: 'user_query_day_hour'})
    data = pd.merge(data, user_query_day_hour, 'left', on=['user_id', 'day', 'hour'])
    ### 统计每天中item展示给user的次数
    user_item_query_day = data.groupby(['item_id', 'user_id', 'day']).size().reset_index()\
                            .rename(columns={0:'user_item_display_day'})
    data = pd.merge(data, user_item_query_day, on=['item_id', 'user_id', 'day'], how='left')
    ###
    user_item_query_hour = data.groupby(['item_id', 'user_id', 'day', 'hour']).size().reset_index()\
                            .rename(columns={0:'user_item_display_hour'})
    data = pd.merge(data, user_item_query_hour, on=['item_id', 'user_id', 'day', 'hour'], how='left')
    # fill na
    data[['user_display_day', 'user_display_day_hour', 'user_item_display_day', 'user_item_display_hour']].fillna(0, inplace=True)
    
    return data

In [4]:
data = pd.read_csv('./data/round1_ijcai_18_train_20180301.txt', sep=' ', encoding='utf-8')
data.head()

,instance_id,item_id,item_category_list,item_property_list,item_brand_id,item_city_id,item_price_level,item_sales_level,item_collected_level,item_pv_level,...,context_page_id,predict_category_property,shop_id,shop_review_num_level,shop_review_positive_rate,shop_star_level,shop_score_service,shop_score_delivery,shop_score_description,is_trade
0,108641074714126964,3412720377098676069,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,4006,5799347067982556520:-1;509660095530134768:-1;5...,6765930309048922341,4,1.0,5002,1.0,1.0,1.0,0
1,5754713551599725161,3412720377098676069,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,4001,5799347067982556520:9172976955054793469;790838...,6765930309048922341,4,1.0,5002,1.0,1.0,1.0,0
2,842679481291040981,3412720377098676069,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,4001,5799347067982556520:5131280576272319091;725801...,6765930309048922341,4,1.0,5002,1.0,1.0,1.0,0
3,937088850059189027,3412720377098676069,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,4016,509660095530134768:-1;5799347067982556520:-1;7...,6765930309048922341,4,1.0,5002,1.0,1.0,1.0,0
4,7975697065017708072,3412720377098676069,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,4001,5799347067982556520:9172976955054793469;790838...,6765930309048922341,4,1.0,5002,1.0,1.0,1.0,0


In [5]:
data = convert_timestamp(data)

D:\Anaconda3\lib\site-packages\pandas\core\frame.py:2754: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


In [6]:
data.head()

,instance_id,item_id,item_category_list,item_property_list,item_brand_id,item_city_id,item_price_level,item_sales_level,item_collected_level,item_pv_level,...,shop_score_delivery,shop_score_description,is_trade,time,day,hour,user_query_day,user_query_day_hour,user_item_query_day,user_item_query_hour
0,108641074714126964,3412720377098676069,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,1.0,1.0,0,2018-09-18 10:09:04,18,10,3,3,1,1
1,5754713551599725161,3412720377098676069,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,1.0,1.0,0,2018-09-18 12:00:32,18,12,8,5,1,1
2,842679481291040981,3412720377098676069,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,1.0,1.0,0,2018-09-18 03:04:12,18,3,2,2,1,1
3,937088850059189027,3412720377098676069,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,1.0,1.0,0,2018-09-18 06:17:50,18,6,1,1,1,1
4,7975697065017708072,3412720377098676069,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,1.0,1.0,0,2018-09-18 19:48:40,18,19,2,2,1,1


In [7]:
len(data)

478138

In [8]:
def feature_group(data, feature, is_trade):
    if (is_trade != 1 and is_trade != 0):
        return None
    else:
        data = data[[feature, 'is_trade']]
        data_feature = data[data['is_trade'] == is_trade]
        data_feature = data_feature.reset_index(drop=True)
        data_feature_grouped = data_feature.groupby([feature]).count()
        data_feature_grouped = data_feature_grouped.reset_index()
        data_feature_grouped = data_feature_grouped.sort_values(['is_trade'], ascending=False)   
    return data_feature_grouped

In [9]:
def feature_log_cal(data, feature, is_trade):
    if (is_trade != 1 and is_trade != 0):
        return None
    else:
        data_feature_grouped = feature_group(data, feature, is_trade)
        new_feature = feature + '_' + str(is_trade)
        data_feature_grouped[new_feature] = np.log(data_feature_grouped['is_trade'] / len(data_feature_grouped))
#         # 0-1标准化
#         data_feature_grouped[new_feature] =  (data_feature_grouped['is_trade'] - data_feature_grouped['is_trade'].min())/\
#                                         (data_feature_grouped['is_trade'].max() - data_feature_grouped['is_trade'].min())
    return data_feature_grouped[[feature, new_feature]]

In [10]:
raw_features = ['item_id', 'item_category_list', 'item_property_list', 'item_brand_id', 'item_city_id', 
                'user_id', 'user_gender_id', 'user_age_level', 'user_occupation_id', 'user_star_level',
               'shop_id', 'context_page_id', 'predict_category_property']

In [11]:
# data_list中存储这每天，每个交易类别的数据
data_list = []
for day in range(18, 25, 1):
    for i in range(2):
        data_day = data[data.day == day]
        data_day_trade = data_day[data_day.is_trade == i]
        data_list.append(data_day_trade)

In [12]:
new_data_list = []
for dl in data_list:
    day = dl.day.unique()[0]
    is_trade_flag = dl.is_trade.unique()[0]
    for feature in raw_features:
        flc = feature_log_cal(dl, feature, is_trade_flag)
        dl = pd.merge(dl, flc, on=feature, how='left')
    new_data_list.append(dl)

In [13]:
new_data_list[0].head()

,instance_id,item_id,item_category_list,item_property_list,item_brand_id,item_city_id,item_price_level,item_sales_level,item_collected_level,item_pv_level,...,item_category_list_0,item_property_list_0,item_brand_id_0,item_city_id_0,user_gender_id_0,user_age_level_0,user_occupation_id_0,user_star_level_0,context_page_id_0,predict_category_property_0
0,108641074714126964,3412720377098676069,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,6.732296,-6.906922,-4.598481,4.354717,8.221681,8.040232,9.172223,6.915558,4.788741,-3.943698
1,5754713551599725161,3412720377098676069,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,6.732296,-6.906922,-4.598481,4.354717,9.582559,7.156783,9.172223,7.003368,7.669052,-7.663349
2,842679481291040981,3412720377098676069,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,6.732296,-6.906922,-4.598481,4.354717,9.582559,8.040232,9.172223,6.737165,7.669052,-9.049643
3,937088850059189027,3412720377098676069,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,6.732296,-6.906922,-4.598481,4.354717,8.221681,7.708760,9.172223,7.003368,3.261935,-4.712353
4,7975697065017708072,3412720377098676069,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,6.732296,-6.906922,-4.598481,4.354717,9.582559,7.156783,9.172223,5.531411,7.669052,-4.174446


In [14]:
print(len(new_data_list))

14


In [15]:
whole_data = pd.DataFrame()
for i in new_data_list:
    whole_data = pd.concat([whole_data, i], axis=0)

In [16]:
len(whole_data)

478138

In [17]:
whole_data.head()

,context_id,context_page_id,context_page_id_0,context_page_id_1,context_timestamp,day,hour,instance_id,is_trade,item_brand_id,...,user_item_query_day,user_item_query_hour,user_occupation_id,user_occupation_id_0,user_occupation_id_1,user_query_day,user_query_day_hour,user_star_level,user_star_level_0,user_star_level_1
0,282924576738839389,4006,4.788741,NaN,1537236544,18,10,108641074714126964,0,1975590437749032870,...,1,1,2005,9.172223,NaN,3,3,3003,6.915558,NaN
1,4007979028023783431,4001,7.669052,NaN,1537243232,18,12,5754713551599725161,0,1975590437749032870,...,1,1,2005,9.172223,NaN,8,5,3006,7.003368,NaN
2,4884875192608989870,4001,7.669052,NaN,1537211052,18,3,842679481291040981,0,1975590437749032870,...,1,1,2005,9.172223,NaN,2,2,3004,6.737165,NaN
3,840119421106178602,4016,3.261935,NaN,1537222670,18,6,937088850059189027,0,1975590437749032870,...,1,1,2005,9.172223,NaN,1,1,3006,7.003368,NaN
4,1736769971710354684,4001,7.669052,NaN,1537271320,18,19,7975697065017708072,0,1975590437749032870,...,1,1,2005,9.172223,NaN,2,2,3001,5.531411,NaN


In [18]:
whole_data.columns

Index(['context_id', 'context_page_id', 'context_page_id_0',
       'context_page_id_1', 'context_timestamp', 'day', 'hour', 'instance_id',
       'is_trade', 'item_brand_id', 'item_brand_id_0', 'item_brand_id_1',
       'item_category_list', 'item_category_list_0', 'item_category_list_1',
       'item_city_id', 'item_city_id_0', 'item_city_id_1',
       'item_collected_level', 'item_id', 'item_price_level',
       'item_property_list', 'item_property_list_0', 'item_property_list_1',
       'item_pv_level', 'item_sales_level', 'predict_category_property',
       'predict_category_property_0', 'predict_category_property_1', 'shop_id',
       'shop_review_num_level', 'shop_review_positive_rate',
       'shop_score_delivery', 'shop_score_description', 'shop_score_service',
       'shop_star_level', 'time', 'user_age_level', 'user_age_level_0',
       'user_age_level_1', 'user_gender_id', 'user_gender_id_0',
       'user_gender_id_1', 'user_id', 'user_item_query_day',
       'user_item_que